# AI Orchestrator - RAG Advance

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## Prep

In [2]:
import os
from openai import AzureOpenAI

MODEL_GPT4 = "gpt-4-turbo"
MODEL_GPT35 = "gpt-35-turbo-1106"
client = AzureOpenAI(
    api_key=os.environ['SC_AOAI_KEY'],
    api_version="2023-12-01-preview",
    azure_endpoint = os.environ['SC_AOAI_ENDPOINT']
)

In [3]:
def _chat_gpt(messages, model=MODEL_GPT35, temp=0.0, topp=0.1):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temp,
        max_tokens=4000,
        top_p=topp
    )   
    
    return response.choices[0].message.content

In [4]:
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os

endpoint = os.environ["AZSCH_ENDPOINT"]
credential = AzureKeyCredential(os.environ["AZSCH_KEY"])
# managed id authentication
#credential = DefaultAzureCredential()

index_name = 'aoai-docs-idx'

In [5]:
# prompt templates
from jinja2 import Template
from common import parse_chat

with open('./determine_reply.jinja2') as file:
    reply_template = file.read()

with open('./determine_intent.jinja2') as file:
    intent_template = file.read()

## Intent

In [6]:
def determine_intent(query, intent_history):
    prompt = Template(intent_template, trim_blocks=True, keep_trailing_newline=True).render(
        intent_history=intent_history,
        query=query
    )

    messages = parse_chat(prompt)
    
    return _chat_gpt(messages)

def extract_intent(input: str, query: str) -> str:
  entries = None
  if 'Single Intents:' in input:
    entries = input.split('Single Intents:', 2)
  elif 'Single Intent:' in input:
    entries = input.split('Single Intent:', 2)
  
  if entries and len(entries) == 2:
    return {
      "current_message_intent": entries[0].strip(),
      "search_intents": entries[1].strip()
    }
  return {
      "current_message_intent": query,
      "search_intents": query
  }

In [7]:
import json

def get_search_intent(query, intent_history):
    raw_intent = determine_intent(query, intent_history)
    intent = extract_intent(raw_intent, query)
    js = json.loads(intent['search_intents'])
    
    return js[0] if len(js) != 0 else "", raw_intent

In [8]:
intent_history = []
query = "What is Assistant API"
intent, raw_intent = get_search_intent(query, intent_history)
intent_history.append(intent)
intent

'what is Assistant API?'

In [9]:
query = "Show some sample code in python"
intent, raw_intent = get_search_intent(query, intent_history)
intent_history.append(intent)
print(intent, "\n", raw_intent)

show sample code for Assistant API in python 
 Show some sample code in python for Assistant API
Single Intents: ["show sample code for Assistant API in python"]


### Search

In [10]:
from aisearch import AISearch
from common import format_retrieved_documents

def get_contenxt(intent):
    search = AISearch(endpoint, index_name, credential)
    docs = search.get_results(intent)
    documentation, metadata = format_retrieved_documents(docs, 4000)

    return documentation, metadata

In [11]:
documentation, metadata = get_contenxt(intent)
documentation

'{\'retrieved_documents\': [{\'[doc0]\': {\'title\': \'assistant.md\', \'content\': \'code."\\n    f"When you are asked to create a visualization you should follow these steps:"\\n    f"1. Write the code."\\n    f"2. Anytime you write new code display a preview of the code to show your work."\\n    f"3. Run the code to confirm that it runs."\\n    f"4. If the code is successful display the visualization."\\n    f"5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",\\n    tools=[{"type": "code_interpreter"}],\\n    model="gpt-4-1106-preview" #You must replace this value with the deployment name for your model.\\n)\\n\\n```\\n\\nThere are a few details you should note from the configuration above:\\n\\n- We enable this assistant to access code interpreter with the line ` tools=[{"type": "code_interpreter"}],`. This gives the model access to a sand-boxed python environment to run and execute code to help 

In [12]:
metadata

[{'title': 'assistant.md',
  'link': '<sup>[1](https://github.com/MicrosoftDocs/azure-docs/tree/main/articles/ai-service/openai/assistant.md)</sup>'},
 {'title': 'assistants-python.md',
  'link': '<sup>[2](https://github.com/MicrosoftDocs/azure-docs/tree/main/articles/ai-service/openai/assistants-python.md)</sup>'},
 {'title': 'assistants-python.md',
  'link': '<sup>[3](https://github.com/MicrosoftDocs/azure-docs/tree/main/articles/ai-service/openai/assistants-python.md)</sup>'}]

### generate answer

In [16]:
def generate_response(intent, documentation):
    prompt = Template(reply_template, trim_blocks=True, keep_trailing_newline=True).render(
        chat_history=chat_history,
        documentation=documentation,
        user_query=intent
    )
    
    messages = parse_chat(prompt)
    response = _chat_gpt(messages)

    return response

In [17]:
chat_history = []
output = generate_response(intent, documentation)

In [18]:
from IPython.display import display, Markdown
display(Markdown(output))

The Assistant API in Python can be used to create an assistant with code interpreter enabled, allowing the model to respond to complex queries by converting questions into code and running that code iteratively in a Python sandbox until it reaches a solution[doc0][doc1][doc2]. Here's a sample code to create an assistant with code interpreter enabled:

```python
import os
import time
import json
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2024-02-15-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

# Create an assistant
assistant = client.beta.assistants.create(
    name="Math Assist",
    instructions="You are an AI assistant that can write code to help answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview" #You must replace this value with the deployment name for your model.
)

# Create a thread
thread = client.beta.threads.create()

# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status

# Wait till the assistant has responded
```
This code demonstrates the creation of an assistant with code interpreter enabled and the process of running a thread to interact with the assistant[doc1][doc2]. Remember to replace the `model` value with the deployment name for your model[doc0][doc1].

In [19]:
query = "assistant api sample code in rest api"
intent, raw_intent = get_search_intent(query, intent_history)
intent_history.append(intent)

print(intent)
documentation = get_contenxt(intent)

output = generate_response(intent, documentation)

assistant api sample code in rest api


In [20]:
print(output)

The sample code for using the Assistant API in REST API can be found in the following snippet:

```console
curl https://YOUR_RESOURCE_NAME.openai.azure.com/openai/assistants?api-version=2024-02-15-preview \
  -H "api-key: $AZURE_OPENAI_KEY" \
  -H "Content-Type: application/json" \
  -d '{
    "instructions": "You are an AI assistant that can write code to help answer math questions.",
    "name": "Math Assist",
    "tools": [{"type": "code_interpreter"}],
    "model": "gpt-4-1106-preview"
  }'
```
This code creates an assistant with specific instructions, name, tools, and model parameters[doc0][doc2]. You can find more details and examples in the Azure OpenAI Assistants REST API documentation[1](https://github.com/MicrosoftDocs/azure-docs/tree/main/articles/ai-service/openai/assistants-rest.md).
